<a href="https://colab.research.google.com/github/pipeton8/6.439-stats-comp-applications/blob/main/Assignments/2%20-%20Genomics/6.439%20Problem%20Set%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem Set 2 - Statistics, Computation and Applications

(c) Felipe del Canto, October 2021
***
***